# Algorytmy tekstowe - odległość edycyjna

In [1]:
import numpy as np
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish

## Dane testowe

In [2]:
tests = [("los", "kloc"),
         ("Łódź", "Lodz"),
         ("kwintesencja", "quintessence"),
         ("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")]

## Algorytm obliczający odległość edycyjną, pozwalający odtworzyć rozwiązanie

In [3]:
def get_editorial_dist(text1, text2):
    n1, n2 = len(text1), len(text2)
    A = [[None] * (n1+1) for _ in range(n2+1)]
    
    # initial edge values
    A[0][0] = (0, None)
    for i in range(1, n1+1):
        A[0][i] = (i, (0, i-1))
    for j in range(1, n2+1):
        A[j][0] = (j, (j-1, 0))
    
    # fill array, with indices of where to go next to get back the solution
    for i in range(1, n1+1):
        for j in range(1, n2+1):
            # if letters are different, add 1
            add = 1
            if text1[i-1] == text2[j-1]:
                add = 0

            x = min(A[j-1][i][0]+1, A[j][i-1][0]+1, A[j-1][i-1][0]+add)
            if A[j-1][i][0] + 1 == x:
                A[j][i] = (x, (j-1, i))
            elif A[j][i-1][0] + 1 == x:
                A[j][i] = (x, (j, i-1))
            else:
                A[j][i] = (x, (j-1, i-1))

    return A, A[-1][-1][0]

## Funkcja wizualizująca w jaki sposób otrzymano rozwiązanie

In [4]:
def get_solution(A, text1, text2):
    solution = [(len(text2), len(text1))]
    while solution[-1] is not None:
        solution.append(A[solution[-1][0]][solution[-1][1]][1])
    
    solution.reverse()
    n = len(solution)
    print(text1)
    for i in range(2, n):
        prev = np.array(solution[i-1])
        curr = np.array(solution[i])
        # no change made
        if np.all(prev + np.array((1, 1)) == curr) and text1[max(curr[1]-1, 0)] == text2[max(curr[0]-1, 0)]:
            continue
        
        # current letter must have been swapped
        if np.all(prev + np.array((1, 1)) == curr):
            print(f"{text2[:max(curr[0]-1, 0)]}*{text2[max(curr[0]-1, 0)]}*{text1[curr[1]:]}")
        
        # letter from text2 must have been inserted
        elif np.all(prev + np.array((1, 0)) == curr):
            print(f"{text2[:max(curr[0]-1, 0)]}*{text2[max(curr[0]-1, 0)]}*{text1[curr[1]:]}")
        
        # letter from text1 must have been deleted
        elif np.all(prev + np.array((0, 1)) == curr):
            print(f"{text2[:max(curr[0], 0)]}**{text1[max(curr[1], 0):]}")

## Wizualizacja na zadanych testach

In [5]:
for i, t in enumerate(tests):
    text1 = t[0]
    text2 = t[1]
    print(f"Test #{i}, text1={text1}, text2={text2}")
    A, swaps = get_editorial_dist(text1, text2)
    get_solution(A, text1, text2)
    print("------------------------------")

Test #0, text1=los, text2=kloc
los
*k*los
klo*c*
------------------------------
Test #1, text1=Łódź, text2=Lodz
Łódź
*L*ódź
L*o*dź
Lod*z*
------------------------------
Test #2, text1=kwintesencja, text2=quintessence
kwintesencja
*q*wintesencja
q*u*intesencja
quintes*s*encja
quintessenc*e*a
quintessence**
------------------------------
Test #3, text1=ATGAATCTTACCGCCTCG, text2=ATGAGGCTCTGGCCCCTG
ATGAATCTTACCGCCTCG
ATGA*G*TCTTACCGCCTCG
ATGAG*G*CTTACCGCCTCG
ATGAGGCT*C*TACCGCCTCG
ATGAGGCTCT*G*CCGCCTCG
ATGAGGCTCTG*G*CCGCCTCG
ATGAGGCTCTGGCC**CCTCG
ATGAGGCTCTGGCCCCT**G
------------------------------


## Najdłuższy wspólny podciąg

In [6]:
def longest_common_sequence(text1, text2):
    n1, n2 = len(text1), len(text2)
    A = [[None] * (n1+1) for _ in range(n2+1)]
    
    # initial edge values
    A[0][0] = 0
    for i in range(1, n1+1):
        A[0][i] = i
    for j in range(1, n2+1):
        A[j][0] = j
    
    # fill array, with indices of where to go next to get back the solution
    for i in range(1, n1+1):
        for j in range(1, n2+1):
            # if letters are different, add 1
            add = 1
            if text1[i-1] == text2[j-1]:
                add = 0

            x = min(A[j-1][i]+1, A[j][i-1]+1)
            if add == 0:
                x = min(x, A[j-1][i-1])
            A[j][i] = x

    return (n1 + n2 - A[-1][-1]) // 2

In [7]:
for i, t in enumerate(tests):
    text1 = t[0]
    text2 = t[1]
    print(f"Test #{i}, text1={text1}, text2={text2}")
    print(f"LCS: {longest_common_sequence(text1, text2)}")
    print("------------------------------")

Test #0, text1=los, text2=kloc
LCS: 2
------------------------------
Test #1, text1=Łódź, text2=Lodz
LCS: 1
------------------------------
Test #2, text1=kwintesencja, text2=quintessence
LCS: 8
------------------------------
Test #3, text1=ATGAATCTTACCGCCTCG, text2=ATGAGGCTCTGGCCCCTG
LCS: 13
------------------------------


## Wczytanie tekstu

In [8]:
text = ""
with open("romeo-i-julia-700.txt", "r", encoding="utf-8") as f:
    text = f.read()

## Tokenizacja Spacy'm

In [9]:
nlp = Polish()
tokenizer = nlp.tokenizer
tokens = tokenizer(text)

In [10]:
# Generate random samples of indexes, which will be used for sampling tokens
size_list = [i for i in range(len(tokens))]
num_sample1 = sorted(np.random.choice(size_list, int(len(tokens)*0.97), replace=False))
num_sample2 = sorted(np.random.choice(size_list, int(len(tokens)*0.97), replace=False))

sample1 = [tokens[i] for i in num_sample1]
sample2 = [tokens[i] for i in num_sample2]

print(f"tokens: {len(tokens)}\nsample1 length: {len(sample1)}\nsample2 length: {len(sample2)}")

tokens: 2699
sample1 length: 2618
sample2 length: 2618


In [11]:
longest_common_sequence(sample1, sample2)

2540

## Diff

In [12]:
def diff_lcs(text1, text2):
    n1, n2 = len(text1), len(text2)
    A = [[None] * (n1+1) for _ in range(n2+1)]
    previous = [[None] * (n1+1) for _ in range(n2+1)]
    
    # initial edge values
    A[0][0] = 0
    for i in range(1, n1+1):
        A[0][i] = i
        previous[0][i] = (0, i-1)
    for j in range(1, n2+1):
        A[j][0] = j
        previous[j][0] = (j-1, 0)
    
    # fill array, with indices of where to go next to get back the solution
    for i in range(1, n1+1):
        for j in range(1, n2+1):
            # if tokens are different, add 1
            add = 1
            if text1[i-1] == text2[j-1]:
                add = 0

            x = min(A[j-1][i]+1, A[j][i-1]+1)
            if add == 0:
                x = min(x, A[j-1][i-1])
            A[j][i] = x
            
            if A[j][i] == A[j-1][i-1] and add == 0:
                previous[j][i] = (j-1, i-1)
            elif A[j][i] == A[j-1][i]+1:
                previous[j][i] = (j-1, i)
            elif A[j][i] == A[j][i-1]+1:
                previous[j][i] = (j, i-1)

    return previous, A[-1][-1]

In [13]:
def diff(text1, text2):
    lines1 = text1.split(sep='\n')
    lines2 = text2.split(sep='\n')
    lcs, changes = diff_lcs(lines1, lines2)
    current = (len(lines2), len(lines1))
    sequence = [current]
    while current is not None:
        current = lcs[current[0]][current[1]]
        sequence.append(current)
    
    sequence.reverse()
    print(f"{changes} differences")
    for i in range(2, len(sequence)):
        if np.all(np.array(sequence[i-1]) + np.array([1, 0]) == np.array(sequence[i])):
            print(f"> {sequence[i][0]}\n{lines2[sequence[i][0]-1]}")
        elif np.all(np.array(sequence[i-1]) + np.array([0, 1]) == np.array(sequence[i])):
            print(f"< {sequence[i][1]}\n{lines1[sequence[i][1]-1]}")

In [14]:
diff("abc\ndef\nghi", "xyz\ndef\nghi\nqwe")

3 differences
< 1
abc
> 1
xyz
> 4
qwe


In [15]:
lines = text.split('\n')
num_sample1 = sorted(np.random.choice([i for i in range(len(lines))], int(len(lines)*0.97), replace=False))
num_sample2 = sorted(np.random.choice([i for i in range(len(lines))], int(len(lines)*0.97), replace=False))

sample1 = "".join([lines[i] + "\n" for i in num_sample1])
sample2 = "".join([lines[i] + "\n" for i in num_sample2])

diff(sample1, sample2)

44 differences
> 15
 * ROMEO — syn Montekiego
> 78
Dalipan, Grzegorzu, nie będziem darli pierza.
> 111
GRZEGORZ
< 116

< 138
GRZEGORZ
< 150
Tym lepiej, że się liczysz do zwierząt; bo gdybyś się liczył do ryb, to byłbyś pewnie sztokfiszem. Weź no się za instrument, bo oto nadchodzi dwóch domowników Montekiego.
< 191

> 192
Skrzywiłeś się na nas, mości panie?
< 247
/ Benwolio ukazuje się w głębi. /
< 260

> 271

< 284

< 288
Przywracam tylko pokój. Włóż miecz nazad
< 299
/ Walczą. Nadchodzi kilku przyjaciół obu partii i mieszają się do zwady; wkrótce potem wchodzą mieszczanie z pałkami. /
> 299

< 323
I szydnie swoją klingą mi urąga.
< 325
/ Wchodzą Monteki i Pani Monteki. /
< 332
/ do żony /
> 334

> 335
/ Wchodzi Książę z orszakiem. /
> 336

> 378

< 423
A chmury — swego oblicza chmurami,
< 431
W czarne bezdroża dusza jego zajdzie,
> 469
Obyś w tej sprawie, co nam serce rani,
> 470
Mógł być szczęśliwszym od nas! Pójdźmy, pani.
> 486

> 528

< 534

> 535
Miłość na oślep zawsze cel swój 